# RA single cell Project: preprocessing and analysis

<br>

**Description**:This Jupyternotebook summarise the single cell pre-processing, the batch correction (Harmonypy) and leiden clustering, the cell subset annotations for the PBMC and each subset\
**Author**: Brenda Miao - Marie Binvignat\
**Version**: 1.0  
**Last updated**: 02/01/23

<br>

**<font size="2">Single Cell Transcriptomics Analysis of Blood Immune Cells Identifies Gene Signatures and Cell Subsets Associated with Disease Activity in a Diverse Population of Rheumatoid Arthritis Patients<font>**
<br>  
<font size="2"> Binvignat M*, Miao B*, Wibrand C*,Yang M, Rychkov D, Flynn E, Nititham J, Khan U, Tamaki W, Carvidi A, Krueger M, Niemi E, Sun Y, Fragiadakis G, Klatzmann D, Sellam J, Mariotti-Ferrandiz E, Gross A, Ye J, Butte AJ, Criswell LA, Nakamura M, Sirota M
 <font><br>
<font size="1"> **equal contribution* <font> 

## Load functions

Loading python pacakges, preprocessing, helping afunctions used for the analysis


In [ ]:
## start here
# get functions
from functions.preprocessing import *
from functions.DEGAnalysis import *

#sns.set_style("ticks")
#sns.set_context("talk")

sns.set(style="ticks")
sns.set_context("notebook")

## Reading in the data

The dataset consisted of 10X single cell RNA sequencing data from PBMC samples taken from 18 early rheumatoid arthritis patients and 18 controls matched by age, sex and ethnicity. Sequencing was performed in 3 batches, with RA and control samples split evenly within batch and lanes. Sample deconvolution and doublet identification was performed using demuxlet and seurat prior to import into scanpy using count, index/column name, and metadata files.



#### Reference for DAS 28 CRP

https://www.rheumatology.org/Portals/0/Files/2019-Update-ACR-Recommended-Rheumatoid-Arthritis-Disease-Activity-Measures.pdf

**2019 update of the American College of Rheumatology recommended rheumatoid arthritis 
disease activity measures**.\
England BR, Tiong BK, Bergman MJ, Curtis JR, Kazi S, Mikuls TR, O'Dell JR, Ranganath VK, Limanni A, Suter LG, Michaud K.  \
*Arthritis care & research. 2019 Dec;71(12):1540-55.*




In [ ]:
### params
adataX = './dataInput/seurat_demux.mtx.gz'
genes = './dataInput/seurat_genes.csv'
barcodes = './dataInput/seurat_barcodes.csv'
metadata = './dataInput/seurat_demux_meta.csv'

outputPath = './dataOutput/adata/seurat_demux.h5ad'

### converting from seurat object to h5ad object (takes a while so only run once)
adata = loadFromSeurat(adataX, genes, barcodes, metadata)

adata.write(outputPath)

"""### Generating metadata file

# read in separate files
outputPath = './dataInput/patientData/RA_metadata_merged.csv'
metadata_RA = pd.read_csv("./dataInput/patientData/RA_patient_DAS_MaryV2.csv")
metadata_demographics = pd.read_csv("./dataInput/patientData/RA_patient_demographics_MaryV2.csv")
metadata_mapping = pd.read_csv("./dataInput/patientData/RA_singlecell_mapping.csv")

# preprocess
metadata_demographics = metadata_demographics.dropna(how = 'all')
del metadata_demographics["dx"]
del metadata_demographics["race"]

# merge and save
meta_test = metadata_mapping.merge(metadata_RA,left_on = "ptnum", right_on = "sample_id", how = "left")
meta_df = meta_test.merge(metadata_demographics,right_on = "subject_id", left_on = "subject_id", how = "left")

# drop duplicates
#meta_df = meta_df[~meta_df["sample"].duplicated()]
meta_df.to_csv(outputPath)

"""

## Preprocessing


In [ ]:
### params
inputPath = './dataOutput/adata/seurat_demux.h5ad'
outputPath = './dataOutput/adata/normalized.h5ad'
plt.rcParams['figure.figsize'] = 4,4
seed = 123
np.random.seed(seed)

# load integrated data
adata = sc.read(inputPath, cache=False)

# clean up data and add in metrics for each cohort
adata = adata[adata.obs["demux.doublet.call"]=="SNG",:]
adata = cleanRAData(adata, plot = True)

# Removing AS patients RA02 with too few cells (68 only) and matching control
adata = adata[adata.obs["sampleType"] != "AS"]
adata = adata[~adata.obs["Sample"].isin(["RA02_1_C_F"])] 
adata = adata[~adata.obs["Sample"].isin(["Control02_1_C_F", "Control11_1_C_M", "Control12_1_C_F","Control13_1_C_M", "Control14_1_C_F"])]

# Remove platelets (> 1 count of platelet/megakaryocyte gene markers (PF4, SDPR, PPBP, GNG11 (not in dataset)))
plt_genes = sc.get.obs_df(adata, keys=["PF4", "GNG11", "PPBP"])
adata = adata[adata.obs.index.isin(plt_genes[plt_genes.sum(axis=1) < 1].index)]

# add metrics for QC and preprocess according to thresholds
adata = plotQCMetrics(adata, groupby="Lane")
adata = set_filters(adata, min_cells=3, min_genes=100, max_genes=1000, min_count=0, max_count=2000, 
               mtpct=20, rbpct=3, remove_rb=False, remove_hb=False, remove_mt=False) 

adata = adata[adata.obs.sort_values("demux_RD_PASS").index]

# Normalize and log-scale. Store raw counts in adata.raw
# NOTE: scaling & regress out performed after select HVG in later cell
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4, key_added="norm_library_size")
sc.pp.log1p(adata) 

# QC plotting
sc.pl.highest_expr_genes(adata, n_top=20);

## Cell cycle scoring
#Remove s and g2m gene not in our dataset ['MLF1IP', 'FAM64A', 'HN1']
s_genes_list = ["MCM5","PCNA", "TYMS","FEN1", "MCM2","MCM4","RRM1","UNG","GINS2","MCM6","CDCA7","DTL","PRIM1","UHRF1", 
                "HELLS","RFC2","RPA2","NASP","RAD51AP1","GMNN","WDR76","SLBP","CCNE2","UBR7","POLD3","MSH2",
                "ATAD2", "RAD51","RRM2","CDC45","CDC6","EXO1","TIPIN","DSCC1",  "BLM","CASP8AP2","USP1","CLSPN","POLA1",
                "CHAF1B","BRIP1" ,"E2F8"]

g2m_genes_list = ["HMGB2","CDK1","NUSAP1","UBE2C","BIRC5","TPX2","TOP2A","NDC80","CKS2","NUF2","CKS1B","MKI67","TMPO",
             "CENPF","TACC3","SMC4","CCNB2","CKAP2L", "CKAP2","AURKB","BUB1","KIF11","ANP32E","TUBB4B",
             "GTSE1","KIF20B","HJURP","CDCA3","CDC20","TTK","CDC25C","KIF2C","RANGAP1","NCAPD2",
             "DLGAP5","CDCA2","CDCA8","ECT2","KIF23","HMMR","AURKA","PSRC1","ANLN","LBR","CKAP5","CENPE","CTCF",
                  "NEK2","G2E3","GAS2L3","CBX5","CENPA"]

sc.tl.score_genes_cell_cycle(adata = adata, s_genes = s_genes_list,g2m_genes = g2m_genes_list )
adata.obs["cell_cycle_diff"] = adata.obs["S_score"] - adata.obs["G2M_score"]

### add metadata
RA_meta = pd.read_csv("./dataInput/patientData/RA_metadata_merged_221118.csv", index_col=0, encoding="ISO-8859-1")

adata.obs = adata.obs[[c for c in adata.obs.columns if c not in RA_meta.columns]]
adata.obs = adata.obs.reset_index(drop=False)

RA_meta['group']= RA_meta['group'].apply(str)
RA_meta["sample_group"] = RA_meta["sample"]+ RA_meta["group"]
adata.obs["sample_group"] = adata.obs["Sample"].astype(str) + adata.obs["batch"].astype(str)
adata.obs = adata.obs.merge(RA_meta, how="left", left_on="sample_group", right_on="sample_group")
adata.obs = adata.obs.set_index("barcodes", drop=True)

# formattting to make adata happy
adata.obs["mtx"] = adata.obs["mtx"].astype(str)
adata.obs["tnf"] = adata.obs["tnf"].astype(str)
adata.obs["early_ra"] = ["Control" if x=="Control"
                         else "True" if y==True 
                         else "False" for x,y in zip(adata.obs["sampleType"],adata.obs["early_ra"])]
adata.obs["acr2010"] = ["Control" if x=="Control"
                         else "True" if y==True 
                         else "False" for x,y in zip(adata.obs["sampleType"],adata.obs["acr2010"])]
adata.obs["acr1987"] = ["Control" if x=="Control"
                         else "True" if y==True 
                         else "False" for x,y in zip(adata.obs["sampleType"],adata.obs["acr1987"])]

# label values
adata.obs["activity_python_crp"] = ["control" if d=="control"
                                else "NA" if np.isnan(r)
                                else "Remission" if r<2.6
                               else "Low" if r<3.2
                               else "Moderate" if r<=5.1
                               else "High" for d, r in zip(adata.obs["dx"], adata.obs["MY_das28crp4"])]

adata.obs["activity_python_binary_crp"] = ["control" if d=="control"
                                       else "NA" if "NA" in r
                                else "No" if "Remission" in r
                               else "No" if "Low" in r
                               else "Yes" if "Moderate" in r
                               else "Yes" for d, r in zip(adata.obs["dx"], adata.obs["activity_python_crp"])]

# label values
adata.obs["activity_python_esr"] = ["control" if d=="control"
                                else "NA" if np.isnan(r)
                                else "Remission" if r<2.6
                               else "Low" if r<3.2
                               else "Moderate" if r<=5.1
                               else "High" for d, r in zip(adata.obs["dx"], adata.obs["MY_das28esr4"])]

adata.obs["activity_python_binary_esr"] = ["control" if d=="control"
                                       else "NA" if "NA" in r
                                else "No" if "Remission" in r
                               else "No" if "Low" in r
                               else "Yes" if "Moderate" in r
                               else "Yes" for d, r in zip(adata.obs["dx"], adata.obs["activity_python_esr"])]


adata.obs["smoker_any"] = ["control" if c=="control"
                           else "Unknown" if type(m)!=str
                           else "No" if "Never" in m
                           else "Yes" for c,m in zip(adata.obs["dx"], adata.obs["smoke_status"]) ]

adata.obs["age_group"] = ["Unknown" if m==np.nan
                          else "Old" if m > 45
                          else "Young" for m in adata.obs["age"]]
adata.obs["age_group"].value_counts()

# print final metrics
print('\nCELLS x GENES = ' + str(adata.shape) + '\n')
    
print('METRICS FOR RAW DATA: \n')
printMetrics(adata, ["sampleType", "sex", "Lane", "Sample"])


# normalized adata has raw values in .raw and log-norm values in .X
adata.write(outputPath)

## Batch correction

In [ ]:
### params
inputPath = './dataOutput/adata/normalized.h5ad'
outputPCs = "./dataOutput/harmony_PCs.csv"
plt.rcParams['figure.figsize'] = 4,4
seed = 123
np.random.seed(seed)

### load integrated data
adata = sc.read(inputPath, cache=False)

### Subset to highly variable genes, justification for values
print(adata.var["mean_counts"].min())
print(adata.var["mean_counts"].mean())
print(adata.var["mean_counts"].max())

# select HVG 
adata.uns['log1p']['base'] = None
sc.pp.highly_variable_genes(adata, flavor='seurat', min_mean=0.01, max_mean=5, min_disp=0.5,batch_key='batch')
sc.pl.highly_variable_genes(adata)
adata = adata[:, adata.var.highly_variable]
print("Highly variable genes: %s"%len(adata.var_names))

# regress out unwanted variables
sc.pp.regress_out(adata, ['total_counts', "n_genes_by_counts", 'pct_counts_mt','pct_counts_ribo', 'cell_cycle_diff'])

# scale data
sc.pp.scale(adata, zero_center = True, max_value=10) 

### dimensional reduction and plotting
sc.pp.pca(adata, n_comps=50)
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50)

# calculate neighbors and UMAP embeddings
sc.pp.neighbors(adata, n_pcs=50, n_neighbors=15)
sc.tl.umap(adata, min_dist=0.1, spread=1)

# plot UMAP
sc.pl.umap(adata, color=['sampleType', 'batch', 'Lane'], wspace=0.35)

# run harmonypy (PREVIOUSLY: 4 iterations, before then included "Lane")
batchCorrect(adata, output_path=outputPCs, max_iter_h=3, vars_use=["batch"]) 

## Clustering and rough and fine subset annotation 



Samples were clustered in an unsupervised manner using leiden clustering and rough annotations were made using a leiden resolution of 3, each cluster were assigned to CD4 T cells,  CD8 T cells, Monocytes, NK cells, B cells.

Marker genes for each cluster were inspected to ensure high fidelity annotations. Platelets, erthryocytes and suspected doublets were removed from further analysis.

Doublets and Rbc-Platelets were excluded from the analysis. 

Subclustering was repeated for each cell type, CD4 T cells, CD8 T cells, B cells, NK cells, and monocytes, with manual annotations based on marker genes and leiden ranked genes

### Cell Dictionnary 

For each cell subset this dictionnary provide the leiden keys, the marker genes, the fine annotation and the color code 

In [ ]:
cells_dict ={"CD4Tcells":{"leiden": 0.8,
                          "markergenes": ["PTPRC", "CCR7", "IL2RB", "SELL", "CD27", "CD28","IL7R","FOXP3",
                                          "CTLA4", "IL10", "LAG3", "TGFB1", "IL2RB", "CD274","CCR7", "IL2", "FAS",
                                          "IFNG", "IL4", "IL5", "GATA3", "STAT4", "TBX21"],
                         "fine_annot": {"0" :"CD4 T Naive", "1" :"CD4 T effector memory", "2" :"CD4 T central memory",
                                   "3" :"CD4 T central memory", "4" : "CD4 T central memory","5" :"CD4 T Naive",
                                   "6" :"Dead_cells","7" :"yd T cells", "8" :"Doublets","9" :"CD4 T central memory",
                                   "10" :"CD4 T IFIT"},
                         "color_subset": [ '#ea4e40','#f37761', '#d33028', '#b3302a', '#72322f'],
                         "color_cmap":"Reds"},
             
             "CD8Tcells":{"leiden": 0.7,
                          "markergenes": ["PTPRC", "CCR7", "SELL", "CD28", "IL7R" , "PECAM1", "TCF7","ITGAL",
                                            "SPN", "CD44", "ITGA4", "IL2RB", "KLRG1", "SELL",
                                            "KLRB1", "GZMH", "NKG7", "FGFBP2", "KLRG1", "PRF1", "FCGR3A","IFNG", "GZMB",
                                            "TIGIT",  "TOX", "IL23R", "RORC", "EOMES", "TBX21"],
                          "fine_annot":{"0":"CD8 TEMRA","1":"CD8 T Naive", "2":"CD8 T early Tem","3":"CD8 T Naive","4":"Doublets","5":"Dead_cells"},
                          "color_subset":['#FFD6A5','#d08c60','#99582a'],
                          "color_cmap":"Oranges"},   
             
             "Monocytes":{"leiden":0.8,
                          "markergenes": ["CD14","IL1B", "IFI6", "IFITM3", "NFKBIA", "NLRP3","FCGR3A","CDKN1C",
                                      "S100A8", "C1QA", "TNF", "IL6", "IL1B", "ITGAM"],
                          "fine_annot":{"0" :"Myeloid DCs","1" :"IL1b-Monocytes","2" :"Classical Monocytes","3": "Classical Monocytes", "4": "Non-classical Monocytes",
                                  "5": "Platelets_RBCs","6": "IFITM3 Monocytes","7": "Doublets","8": "Doublets","9": "Doublets", "10": "Doublets", "11":"Myeloid DCs"},
                          "color_subset":['#afaed4','#8d89c0','#705eaa', '#7209b7','#3c096c'],
                          "color_cmap":"Purples"},
             
             "NKcells":{"leiden":0.5,
                        "markergenes":["NKG7", "NCAM1", "KLRD1" , "NCR1", "KLRB1", "GNLY", "CCR7", "NCR1"],
                        "fine_annot":{"0" :"NKCD56bright", "1" :"NKCD56bright",  "2" :"NKCD56low", "3": "Doublets"},
                        "color_subset":['#8ed08b', '#2c944c'],
                        "color_cmap":"Greens"},
             
             "Bcells":{"leiden":0.8,
                       "markergenes":["CD79A", "CD19", "CD27", "CD38", "CD7", "ITGAM", "CD38", "MS4A1", "ITGA4", "IGHD", "IGKC", "IGHM"],
                       "fine_annot": {"0" :"Naive Bcells", "1" :"Memory Bcells", "2" :"Naive Bcells", "3": "Memory Bcells",
                                     "4": "Plasmablasts","5": "Plasmablasts","6": "Platelets_RBCs","7": "Doublets","8": "Doublets"},
                       "color_subset":['#2a6f97','#4cc9f0','#023e8a','#0267c1',  '#A0C4FF'],
                       "color_cmap": "Blues"}}

### Overall Leiden clustering 

Computing the leiden clustering for the global adata, the leiden resolution used is 3.0

In [ ]:
### params
pcaPath = "./dataOutput/harmony_PCs.csv"
inputPath = './dataOutput/adata/normalized.h5ad'
outputPath = './dataOutput/adata/clustered.h5ad'
plt.rcParams['figure.figsize'] = 4,4
leiden_key = "leiden"
rank_key = "leiden_markers"
seed = 123
np.random.seed(seed)

### read input
# normalized adata has raw values in .raw and log-norm values in .X
adata = sc.read(inputPath, cache=False)
adata.uns['log1p']['base'] = None

# load corrected PCs
PCA_df = pd.read_csv(pcaPath, index_col=0)
adata.obsm['X_pca'] = PCA_df.values

# calculate neighbors and UMAP embeddings
sc.pp.neighbors(adata, n_pcs=50, n_neighbors=15) 
sc.tl.umap(adata, min_dist=0.1, spread=1)

# plot, with normalized data stored in raw values
sc.pl.umap(adata, color=['sampleType', 'batch', 'Lane'], wspace=0.35)
sc.pl.umap(adata, ncols=4, color=["CD2", "CD3D", "CD8A","CD4",
                         "CD33", "ITGAM","CD14", "FCGR3A",
                        "NKG7", "NCAM1", "CD79A",  "CD19"], size=3, vmax=5, use_raw=True)

### leiden params
leiden_key = "leiden"
rank_key = "leiden_markers"
groupby = "leiden_r3.0"

# calculate leiden 
sc.tl.leiden(adata, resolution=3.0, key_added=leiden_key+"_r3.0")

# plot leiden & marker genes
sc.pl.umap(adata, color=[groupby])

### rank genes groups
adata.uns['log1p']['base'] = None
sc.tl.rank_genes_groups(adata, groupby=groupby, use_raw=True, pts=True, key_added=rank_key, method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, key=rank_key, use_raw = False, ncols=6)
sc.pl.rank_genes_groups_dotplot(adata, n_genes=5, key=rank_key, groupby=groupby, use_raw = False)

 # histplots
plt.rcParams['figure.figsize'] = [40, 5]
sc.pl.violin(adata, ['n_genes_by_counts'],jitter=0.1, groupby = groupby, rotation= 90,cut=0) 
sc.pl.violin(adata, ['pct_counts_mt'],jitter=0.1, groupby = groupby, rotation= 90,cut=0) 
sc.pl.violin(adata, ['pct_counts_ribo'],jitter=0.1, groupby = groupby, rotation= 90,cut=0) 
sc.pl.violin(adata, ['pct_counts_hb'],jitter=0.1, groupby = groupby, rotation= 90,cut=0) 

### save clustered anndat
adata.write(outputPath)

### sc.pl.umap(adata, color=['sampleType', 'batch', 'Lane'], wspace=0.35)
plt.rcParams['figure.figsize'] = 5,5
cluster_small_multiples(adata, clust_key="leiden_r3.0", ncols =6)

### Rough annotation and creation of cell subset adata and leiden clustering

In [ ]:
### Compositional analysis on unannotated clusters
# params
plt.rcParams['figure.figsize'] = [5, 5]
inputPath = './dataOutput/adata/clustered.h5ad'
outputPath = './dataOutput/adata/roughannot.h5ad'
rank_key = "leiden_markers"
groupby = "leiden_r3.0"
leiden_key = "leiden_subset"
seed = 123
np.random.seed(seed)

# load adata  
# normalized adata has raw values in .raw and log-norm values in .X
adata = sc.read(inputPath, cache=False)

# plotting
#sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, key=rank_key, use_raw = False, ncols=6)
#sc.pl.rank_genes_groups_dotplot(adata, n_genes=5, key=rank_key, groupby=groupby, use_raw = False)
# remove RBCs (high hemoglobin gene expression, group 14) and platelets (NRGN, TUBB1, group 9)
#adata = adata[~adata.obs[groupby].isin(["14", "9"])]

# rough annotations
adata.obs["rough_annot"] = ["Monocytes" if c in ["6","7","10","22","25","28","29","30","33","39", "40"] #CD14+/CD16+ 
                            else "CD4Tcells" if c in ["1","2","4","8","11","12","13","15","16","19","24","26","27","32","34","36","37"]
                            else "CD8Tcells" if c in ["9","14"]
                            else "NKcells" if c in ["0", "23"]
                            else "Bcells" if c in ["3", "5", "20","35","38"]  
                            else "Platelets_RBCs" if c in ["17","31","41"]
                            else "Doublets" if c in ["18","21"]
                            else "Dead_cells" if c in ["42", "43"]
                            else "NA" for c in adata.obs[groupby]]

sc.pl.umap(adata, color=["rough_annot"])

# plot proportion of cells in each sample
tmp = pd.crosstab(adata.obs['Sample'],adata.obs['rough_annot'], normalize='index')
tmp.plot.bar(stacked=True, figsize=(15,6)).legend(loc='center', bbox_to_anchor=(1, 0., 0.5, 0.5))

# Remove dead cells, hb and platelets
adata =adata[~(adata.obs["rough_annot"].str.contains("Platelets_RBCs|Doublets|Dead_cells")),:]
sc.pl.umap(adata, color=["rough_annot"])

adata.uns['log1p']['base'] = None
adata.write(outputPath)

### Creation of the subset adata
for subset in cells_dict:
    
    seed = 123
    np.random.seed(seed)

    print(subset)
    
    # get subset
    adata_sub = adata[adata.obs["rough_annot"] == subset]

    print(subset)
    print(cells_dict[subset]["leiden"])

    # values for HVG cutoffs values
    print(adata_sub.var["mean_counts"].min())
    print(adata_sub.var["mean_counts"].mean())
    print(adata_sub.var["mean_counts"].max())

    ### batch correct using HarmonyPy
    # show UMAP before batch correction
    # #TODO - check if need to scale/regress out again

    #calculateUMAP(adata, flavor='seurat', min_mean=0.0125, max_mean=3, min_disp=0.5, hvg_batch='batch',
    #n_neighbors=10, n_pcs=60, return_full=False, umap=False):

    adata_umap = calculateUMAP(adata_sub, return_full=False, n_pcs=40,min_mean=0.01, max_mean=4, min_disp=0.5,  
                                   hvg_batch="batch", umap=False, n_neighbors=10) 

    sc.pl.umap(adata_sub, color=['sampleType', 'batch', 'Lane'], wspace=0.35)

    # batch correct
    pc_df = batchCorrect(adata_umap, output_path=None, max_iter_h=3, vars_use=["batch"])
    adata_sub.obsm['X_pca'] = pc_df
    sc.pl.umap(adata_sub, color=['sampleType', 'batch', 'Lane'], wspace=0.35)

    # calculate updated UMAP
    sc.pp.neighbors(adata_sub, n_pcs=40, n_neighbors=10)
    sc.tl.umap(adata_sub, min_dist=0.1, spread=1)
    sc.pl.umap(adata_sub, color=['dx', 'batch', 'Lane'], wspace=0.35)

    #### leiden clustering
    sc.tl.leiden(adata_sub, resolution=cells_dict[subset]["leiden"], key_added=leiden_key+"_r"+str(cells_dict[subset]["leiden"]))

    # fine annotations
    sc.tl.rank_genes_groups(adata_sub, groupby=leiden_key+"_r"+str(cells_dict[subset]["leiden"]), use_raw=False, pts=True, 
                            key_added=leiden_key+"_rank", method='wilcoxon')

    # Embedding density (Activity score)
    #sc.tl.embedding_density(adata_sub, basis='umap', groupby="activity_python_binary")

    ### save to final file
    adata_sub.write("./dataOutput/subset_adata/" + subset +"_normalized.h5ad")
    
    

### Fine annotation of subsets clusters

In [ ]:
### params
plt.rcParams['figure.figsize'] = [5, 5]
rough_annot = "rough_annot"
leiden_key = "leiden_subset"
global_markers_genes = ["CD2", "CD3D", "CD8A","CD4","CD33", "ITGAM","CD14", "FCGR3A", "NKG7", "NCAM1", "CD79A",  "CD19"]
hue_groups = ["control", "No", "Yes"] #, ["Positive", "Negative"] #["control", "No", "Yes"]
hue = "activity_python_binary_crp" #"MY_MTX" #"dx" #"activity_python_binary"


for subset in cells_dict:
    
    print(subset)
    seed = 123
    np.random.seed(seed)
    
    adata_sub = sc.read_h5ad("./dataOutput/subset_adata/" + subset +"_normalized.h5ad")
    adata_sub.uns['log1p']['base'] = None
    
    marker_genes_subset = cells_dict[subset]["markergenes"]
    
    #subset leiden clustering
    sc.pl.umap(adata_sub, color=leiden_key+"_r"+str(cells_dict[subset]["leiden"]))
    sc.tl.rank_genes_groups(adata_sub, groupby=leiden_key+"_r"+str(cells_dict[subset]["leiden"]), use_raw=False, pts=True, key_added=leiden_key+"_rank", method='wilcoxon')
    sc.pl.rank_genes_groups(adata_sub, n_genes=25, sharey=False, key=leiden_key+"_rank", use_raw = False, ncols=6)
    sc.pl.rank_genes_groups_dotplot(adata_sub, n_genes=10, key=leiden_key+"_rank", groupby=leiden_key+"_r"+str(cells_dict[subset]["leiden"]), use_raw = False)
    
    #subset leiden cluster
    cluster_small_multiples(adata_sub, clust_key=leiden_key+"_r"+str(cells_dict[subset]["leiden"]))
    
    #marker genes
    sc.pl.umap(adata_sub, color = global_markers_genes+marker_genes_subset, size = 50)

    #Violin plot and  subset cluster quality control
    plt.rcParams['figure.figsize'] = [40, 5]
    sc.pl.violin(adata_sub, ['n_genes_by_counts'],jitter=0.1, groupby = leiden_key+"_r"+str(cells_dict[subset]["leiden"]), rotation= 90,cut=0) 
    sc.pl.violin(adata_sub, ['pct_counts_mt'],jitter=0.1, groupby = leiden_key+"_r"+str(cells_dict[subset]["leiden"]), rotation= 90,cut=0) 
    sc.pl.violin(adata_sub, ['pct_counts_ribo'],jitter=0.1, groupby = leiden_key+"_r"+str(cells_dict[subset]["leiden"]), rotation= 90,cut=0) 
    sc.pl.violin(adata_sub, ['pct_counts_hb'],jitter=0.1, groupby = leiden_key+"_r"+str(cells_dict[subset]["leiden"]), rotation= 90,cut=0) 
    plt.rcParams['figure.figsize'] = [5, 5]
    
    ##### Subset annotation
    adata_sub.obs["fine_annot"] = adata_sub.obs[leiden_key+"_r"+str(cells_dict[subset]["leiden"])].map(cells_dict[subset]["fine_annot"])
    sc.pl.umap(adata_sub, color="fine_annot")
    
    # load color palette
    '''colors = {"Bcells":("Blues", 3), "CD4Tcells":("Reds", 5), "CD8Tcells":("pink_r", 4),
         "Monocytes":("Purples", 5), "NKcells":("Greens", 3)}
    subset_colors = []

    for c in colors.keys():
    '''
    
    #Removing platelettes and doublets
    adata_sub =adata_sub[~(adata_sub.obs["fine_annot"].str.contains("Dead_cells|Platelets_RBCs|Doublets")),:]
    
    adata_sub = adata_sub[:,~adata_sub.var_names.str.startswith('MT-')]
    #Should we remove RB genes? in the DEG and in rank genes
    adata_sub = adata_sub[:,~adata_sub.var_names.str.startswith('RPL')]
    adata_sub = adata_sub[:,~adata_sub.var_names.str.startswith('RPS')]
    adata = adata[:,~adata.var_names.str.startswith('MRPL')]
    adata = adata[:,~adata.var_names.str.startswith('MRPS')]
    
    sc.tl.rank_genes_groups(adata_sub, groupby="fine_annot", use_raw=False, pts=True, 
                        key_added=leiden_key+"_rank", method='wilcoxon')
    

    sc.pl.umap(adata_sub, color="fine_annot", palette = cells_dict[subset]["color_subset"])
    sc.pl.rank_genes_groups(adata_sub, key=leiden_key+"_rank", n_genes=15, sharey=False, use_raw = False, ncols=4 )
    sc.pl.rank_genes_groups_dotplot(adata_sub, n_genes=3, key=leiden_key+"_rank", groupby="fine_annot", use_raw = False, cmap = cells_dict[subset]["color_cmap"])  


    ### Subset analysis
       
    adata_sub.write("./dataOutput/subset_adata/" + subset +"_clustered.h5ad")    

###  Final adata with subsets clustering 

In [ ]:
### Merging fine annot
plt.rcParams['figure.figsize'] = [5, 5]
inputPath = './dataOutput/adata/roughannot.h5ad'
outputPath = './dataOutput/adata/final.h5ad'

adata = sc.read_h5ad(inputPath)

adata = adata[:,~adata.var_names.str.startswith('MT-')]
adata = adata[:,~adata.var_names.str.startswith('RPL')]
adata = adata[:,~adata.var_names.str.startswith('RPS')]
adata = adata[:,~adata.var_names.str.startswith('MRPL')]
adata = adata[:,~adata.var_names.str.startswith('MRPS')]
    

fine_annot_dict = {}

for subset in cells_dict:
    print(subset)
    adata_sub = sc.read_h5ad("./dataOutput/subset_adata/" + subset +"_clustered.h5ad")
    fine_annot_sub = dict(zip(adata_sub.obs.index, adata_sub.obs["fine_annot"]))
    fine_annot_dict.update(fine_annot_sub)

adata.obs["fine_annot"] = adata.obs.index.map(fine_annot_dict)

# reorder values
#adata.obs[fine_annot] = adata.obs[final_annot].astype("category")
#adata.obs[fine_annot].cat.reorder_categories(cell_subsets, inplace=True)
#sc.pl.umap(adata,color = fine_annot, palette = sns.color_palette(subset_colors), title="", size=3)

#adata.obs["fine_color"] = adata.obs.index.map(fine_color_dict)

adata = adata[adata.obs["fine_annot"].notnull()]

adata.obs["fine_annot"] = adata.obs["fine_annot"].astype("category")

adata.obs['fine_annot'].cat.reorder_categories(
['CD4 T central memory','CD4 T effector memory','CD4 T IFIT','CD4 T Naive','yd T cells','CD8 T early Tem',
'CD8 T Naive','CD8 TEMRA','Classical Monocytes','IFITM3 Monocytes',
 'IL1b-Monocytes','Myeloid DCs','Non-classical Monocytes','NKCD56bright','NKCD56low',
 'Memory Bcells', 'Naive Bcells','Plasmablasts'], inplace = True)

color_subset = ['#ea4e40','#f37761', '#d33028', '#b3302a', '#72322f',
                '#FFD6A5','#d08c60','#99582a',
                '#afaed4','#8d89c0','#705eaa', '#7209b7','#3c096c',
                '#8ed08b', '#2c944c',
                '#2a6f97','#4cc9f0','#023e8a','#0267c1',  '#A0C4FF']
                
sc.pl.umap(adata, color = "fine_annot", palette = color_subset)

adata.write(outputPath)